## Loading the dataset

# Training Armenian tokenizer

## Getting a corpus

In [1]:
from datasets import load_dataset
dataset = load_dataset("Karavet/ARPA-Armenian-Paraphrase-Corpus")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


For this example, we will use ARPA-Armenian-Paraphrase-Corpus (which contains 5000 rows of text)

In [2]:
first_row = dataset['train'][0]

To access an element, we just have to provide its index:

In [3]:
first_row

{'Sentence1': 'Այդ մասին ասվում է Վրաստանի 2019թ. միգրացիոն տեղեկատուում, որը հրապարակվել է միգրացիայի կառավարական հանձնաժողովի կայքում:',
 'Sentence2': 'Այդպիսով, Հայաստանը 3-րդ տեղում է այն երկրների ցանկում, որոնց քաղաքացիները նշված ժամանակահատվածում կացության իրավունք են ստացել Վրաստանում:',
 'Paraphrase': -1}

In [4]:
dataset['train'][:5]

{'Sentence1': ['Այդ մասին ասվում է Վրաստանի 2019թ. միգրացիոն տեղեկատուում, որը հրապարակվել է միգրացիայի կառավարական հանձնաժողովի կայքում:',
  'Լուսանկարի օրիգինալը պահվում է Նորվեգիայի Թագավորական արխիվում:',
  'Այս մասին հայտնում է ռուսաստանյան «ՌԻԱ Նովոստի» գործակալությունը:',
  'Զվարթնոցում» ազգականիս դիմավորելիս նկատեցի, որ ընդամենը շորով փակվել է ցանկապատի այն հատվածը, որտեղից երևում էր աղբանոցը»,- նշում է Զարուհի Մուրադյանը:',
  'Վարձակալելով օդանավը՝ «Արմավիան» դրան տվեց «Մեսրոպ Մաշտոց» անունը:'],
 'Sentence2': ['Այդպիսով, Հայաստանը 3-րդ տեղում է այն երկրների ցանկում, որոնց քաղաքացիները նշված ժամանակահատվածում կացության իրավունք են ստացել Վրաստանում:',
  'Բնօրինակ լուսանկարը պահվում է Նորվեգիայի Թագավորական արխիվում:',
  'Դեպքի առնչությամբ հարցաքննվել է 4 մարդ՝ այդ թվում նաև առևտրի կենտրոնի վարձակալ ընկերության ղեկավարը, որի տարածքում է գտնվել կրակի օջախը:',
  'Զվարթնոցում իմ ազգականին ողջունելիս նկատեցի, որ աղբավայրը նայող ցանկապատը փակվել է միայն շորով », - ասում է Զարուհի Մու

In [5]:
len(dataset['train'])

4233

To avoid loading everything into memory (since the Datasets library keeps the element on disk and only load them in memory when requested), we define a Python iterator. This is particularly useful if you have a huge dataset:

In [6]:
def batch_iterator(batch_size=100):
    for i in range(0, len(dataset['train']), batch_size):
        # Extracting sentences from the dataset
        batch_sentences = dataset['train']['Sentence1'][i:i+batch_size] + dataset['train']['Sentence2'][i:i+batch_size]
        yield batch_sentences

Now let's see how we can use this corpus to train a new tokenizer! There are two APIs to do this: the first one uses an existing tokenizer and will train a new version of it on your corpus in one line of code, the second is to actually build your tokenizer block by block, so lets you customize every step!

## Using an existing tokenizer

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

Make sure that the tokenizer you picked as a *fast* version (backed by the 🤗 Tokenizers library) otherwise the rest of the notebook will not run:

In [8]:
tokenizer.is_fast

True

Then we feed the training corpus (either the list of list or the iterator we defined earlier) to the `train_new_from_iterator` method. We also have to specify the vocabulary size we want to use:

In [9]:
arm_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=25000)

In [10]:
arm_tokenizer(dataset['train'][:5]["Sentence1"])

{'input_ids': [[670, 468, 1665, 284, 1934, 2274, 360, 14, 7975, 904, 2622, 281, 12, 636, 2788, 284, 21666, 5492, 2362, 4276, 26], [20328, 22023, 7558, 284, 15580, 15201, 10326, 26], [664, 468, 689, 284, 6520, 406, 13589, 7032, 350, 4702, 26], [20653, 350, 19398, 388, 11226, 388, 18723, 12, 323, 3908, 16655, 7833, 284, 18230, 264, 513, 6004, 12, 1077, 355, 5280, 454, 1616, 19150, 571, 2352, 284, 13636, 14978, 26], [676, 452, 814, 591, 14276, 372, 406, 12206, 350, 2624, 5917, 406, 404, 24720, 15342, 350, 3137, 26]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [11]:
arm_tokenizer

GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=25000, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

Encoding sample text

In [12]:
sample_text = "Նշվում է, որ նա կռվի է բռնվել Դաղստանից 32-ամյա Արթուր Ռասուլովի հետ"


encoded_input = arm_tokenizer.encode(sample_text, add_special_tokens=True)

print("Token IDs:", encoded_input)

Token IDs: [2117, 284, 12, 323, 425, 16052, 613, 284, 3440, 380, 724, 349, 410, 952, 3195, 13, 1553, 2427, 521, 294, 611, 1619, 385]


Decoding sample text back

In [13]:
# Decode the token IDs back to tokens
tokens = [arm_tokenizer.decode([token_id]) for token_id in encoded_input]

# Print the tokens
print("Tokens:", tokens)

Tokens: ['Նշվում', ' է', ',', ' որ', ' նա', ' կռ', 'վի', ' է', ' բռն', 'վել', ' Դ', 'աղ', 'ստ', 'անից', ' 32', '-', 'ամյա', ' Արթուր', ' Ռ', 'աս', 'ուլ', 'ովի', ' հետ']


Saving tokenizer

In [14]:
arm_tokenizer.save_pretrained("Armenian-tokenizer")

('Armenian-tokenizer/tokenizer_config.json',
 'Armenian-tokenizer/special_tokens_map.json',
 'Armenian-tokenizer/vocab.json',
 'Armenian-tokenizer/merges.txt',
 'Armenian-tokenizer/added_tokens.json',
 'Armenian-tokenizer/tokenizer.json')

In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [16]:
arm_tokenizer = AutoTokenizer.from_pretrained("Armenian-tokenizer")

In [17]:
dataset = load_dataset("oscar-corpus/OSCAR-2201",
                        use_auth_token=True,
                        language="hy",
                        streaming=True,
                        split="train")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for oscar-corpus/OSCAR-2201 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar-corpus/OSCAR-2201
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [18]:
dataset

IterableDataset({
    features: ['id', 'text', 'meta'],
    n_shards: 5
})

In [19]:
for i, data in enumerate(dataset):
    print(data)
    if i == 50:  
        break

{'id': 0, 'text': 'Private BankingԱրտոնությունԲիզնեսինՆերդրումային բիզնեսՎՏԲ-Հայաստան Բանկի մասինԲաժնետերեր և ներդրողներՎՏԲ խմբի մասին\nՎարկեր\nՎարկի գնում\nՀիփոթեք\nԱվտովարկ\nՍպառողական վարկեր\nԱպառիկ վարկավորում\nԳյուղատնտեսական վարկ\nԱվանդներ\nՇահավետ ժամկետային ավանդ\nՊրոգրես ժամկետային ավանդ\nՀարմարավետ ժամկետային ավանդ\nՍոցիալական ժամկետային ավանդ\nՀաշիվներ\nՑպահանջ հաշիվներ\nՀատուկ հաշիվներ\nԿուտակային կենսաթոշակային համակարգ\nՔարտեր\nԴեբետային քարտեր\nՎարկային քարտեր\nԱկնթարթային տրամադրման քարտեր\nՆպատակային քարտեր\nՀեռահար սպասարկում\nՄոբայլ/Ինտերնետ բանկինգ\nSMS բանկինգ\nՀեռախոս բանկ\n«Բանկն աշխատավայրում» ծրագիր\nԳործարքներ բանկոմատներով\nՎճարային տերմինալներ և էլեկտրոնային դրամապանակներ\nԿոմունալ վճարումներ հեռակա եղանակով\nՎիդեո բանկինգ\nՉատ-Բանկ DIRECT\nԱկցիաներ\nՖիզիկական անձանց S.W.I.F.T. փոխանցումներ դեպի ՌԴ` իջեցված սակագներով\nԱվանդային ակցիա «Ամառային» New\nԱվանդային ակցիա «Խնայողություների օր» New\nՇահավետ պայմաններով ոսկյա իրերի գրավադրմամբ վարկերի ակցիա New\nԱկց

In [20]:
def tokenize_function(examples):
    return arm_tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

In [21]:
special_tokens_dict = {
    'eos_token': '[EOS]',
    'bos_token': '[BOS]',
    'unk_token': '[UNK]',
    'pad_token': '[PAD]',
    'sep_token': '[SEP]',
    'cls_token': '[CLS]',
    'mask_token': '[MASK]'
}
arm_tokenizer.add_special_tokens(special_tokens_dict)


7

In [22]:
#arm_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#model.resize_token_embeddings(len(arm_tokenizer))

In [23]:
sample = next(iter(dataset))
tokenized_sample = tokenize_function(sample)

In [24]:
# Print the original text and its tokenized form
print("Original text:\n", sample['text'][:300])

Original text:
 Private BankingԱրտոնությունԲիզնեսինՆերդրումային բիզնեսՎՏԲ-Հայաստան Բանկի մասինԲաժնետերեր և ներդրողներՎՏԲ խմբի մասին
Վարկեր
Վարկի գնում
Հիփոթեք
Ավտովարկ
Սպառողական վարկեր
Ապառիկ վարկավորում
Գյուղատնտեսական վարկ
Ավանդներ
Շահավետ ժամկետային ավանդ
Պրոգրես ժամկետային ավանդ
Հարմարավետ ժամկետային ավանդ
Սոց


In [25]:
print("\nTokenized output:\n", tokenized_sample)


Tokenized output:
 {'input_ids': [48, 82, 6675, 2370, 69, 1716, 1389, 23122, 3879, 24877, 587, 3200, 12813, 16731, 1081, 373, 6062, 676, 773, 587, 13, 5526, 13239, 468, 10335, 276, 276, 330, 2498, 10577, 305, 237, 773, 587, 1582, 468, 199, 6890, 276, 199, 676, 2391, 2894, 199, 364, 6691, 1345, 543, 199, 14669, 313, 423, 199, 15118, 1602, 7755, 199, 6200, 16463, 1504, 2537, 199, 5855, 5355, 1504, 199, 3746, 542, 301, 199, 1376, 16300, 3497, 373, 3184, 199, 608, 260, 782, 7282, 3497, 373, 3184, 199, 364, 5514, 2087, 3497, 373, 3184, 199, 528, 1175, 1967, 3497, 373, 3184, 199, 5392, 17145, 199, 2050, 3477, 948, 6954, 301, 199, 20822, 6954, 301, 199, 479, 614, 4182, 8992, 1108, 199, 1025, 449, 276, 199, 527, 1963, 5747, 1573, 276, 199, 6890, 373, 1573, 276, 199, 2578, 4764, 1455, 373, 11483, 1573, 276, 199, 16733, 373, 1573, 276, 199, 6221, 1557, 3932, 8298, 199, 404, 1113, 1137, 15, 4857, 18143, 1816, 1253, 199, 51, 22996, 1816, 1253, 199, 6221, 2260, 1816, 199, 399, 587, 535, 263, 564, 

In [26]:
first_50_ids = tokenized_sample['input_ids'][:50]
first_50_tokens = [arm_tokenizer.decode([tid]) for tid in first_50_ids]

In [27]:
print("First 50 token IDs:", first_50_ids)

First 50 token IDs: [48, 82, 6675, 2370, 69, 1716, 1389, 23122, 3879, 24877, 587, 3200, 12813, 16731, 1081, 373, 6062, 676, 773, 587, 13, 5526, 13239, 468, 10335, 276, 276, 330, 2498, 10577, 305, 237, 773, 587, 1582, 468, 199, 6890, 276, 199, 676, 2391, 2894, 199, 364, 6691, 1345, 543, 199, 14669]


In [28]:
print("First 50 tokens:", first_50_tokens)

First 50 tokens: ['P', 'r', 'iv', 'at', 'e', ' B', 'an', 'king', 'Արտ', 'ոնություն', 'Բ', 'իզն', 'եսին', 'Ներդ', 'րում', 'ային', ' բիզնես', 'Վ', 'Տ', 'Բ', '-', 'Հայաստան', ' Բանկի', ' մասին', 'Բաժնետ', 'եր', 'եր', ' և', ' ներդր', 'ողն', 'եր�', '�', 'Տ', 'Բ', ' խմբի', ' մասին', '\n', 'Վարկ', 'եր', '\n', 'Վ', 'արկի', ' գնում', '\n', 'Հ', 'իփ', 'ոթ', 'եք', '\n', 'Ավտ']


In [29]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [30]:
for i, data in enumerate(tokenized_dataset):
    print(data)
    if i == 50:
        break

{'id': 0, 'text': 'Private BankingԱրտոնությունԲիզնեսինՆերդրումային բիզնեսՎՏԲ-Հայաստան Բանկի մասինԲաժնետերեր և ներդրողներՎՏԲ խմբի մասին\nՎարկեր\nՎարկի գնում\nՀիփոթեք\nԱվտովարկ\nՍպառողական վարկեր\nԱպառիկ վարկավորում\nԳյուղատնտեսական վարկ\nԱվանդներ\nՇահավետ ժամկետային ավանդ\nՊրոգրես ժամկետային ավանդ\nՀարմարավետ ժամկետային ավանդ\nՍոցիալական ժամկետային ավանդ\nՀաշիվներ\nՑպահանջ հաշիվներ\nՀատուկ հաշիվներ\nԿուտակային կենսաթոշակային համակարգ\nՔարտեր\nԴեբետային քարտեր\nՎարկային քարտեր\nԱկնթարթային տրամադրման քարտեր\nՆպատակային քարտեր\nՀեռահար սպասարկում\nՄոբայլ/Ինտերնետ բանկինգ\nSMS բանկինգ\nՀեռախոս բանկ\n«Բանկն աշխատավայրում» ծրագիր\nԳործարքներ բանկոմատներով\nՎճարային տերմինալներ և էլեկտրոնային դրամապանակներ\nԿոմունալ վճարումներ հեռակա եղանակով\nՎիդեո բանկինգ\nՉատ-Բանկ DIRECT\nԱկցիաներ\nՖիզիկական անձանց S.W.I.F.T. փոխանցումներ դեպի ՌԴ` իջեցված սակագներով\nԱվանդային ակցիա «Ամառային» New\nԱվանդային ակցիա «Խնայողություների օր» New\nՇահավետ պայմաններով ոսկյա իրերի գրավադրմամբ վարկերի ակցիա New\nԱկց

In [33]:
import torch
from itertools import islice

def stream_dataloader(stream_dataset, tokenizer, batch_size=8):
    iterator = iter(stream_dataset)  # Ensure stream_dataset is iterable
    while True:
        batch = list(islice(iterator, batch_size))
        if not batch:
            break

        # Process the batch
        batch_input_ids = []
        batch_attention_mask = []
        for item in batch:
            encoded_item = tokenizer(item['text'], truncation=True, padding="max_length", max_length=512, return_tensors="pt")
            batch_input_ids.append(encoded_item['input_ids'])
            batch_attention_mask.append(encoded_item['attention_mask'])

        # Concatenate all tensors in the batch
        batch_input_ids = torch.cat(batch_input_ids, dim=0)
        batch_attention_mask = torch.cat(batch_attention_mask, dim=0)

        yield {'input_ids': batch_input_ids, 'attention_mask': batch_attention_mask}


In [34]:
first_batch = stream_dataloader(tokenized_dataset, arm_tokenizer, batch_size=1)

In [35]:
first_batch

<generator object stream_dataloader at 0x79daa69c3300>

In [36]:
next(first_batch)


{'input_ids': tensor([[   48,    82,  6675,  2370,    69,  1716,  1389, 23122,  3879, 24877,
            587,  3200, 12813, 16731,  1081,   373,  6062,   676,   773,   587,
             13,  5526, 13239,   468, 10335,   276,   276,   330,  2498, 10577,
            305,   237,   773,   587,  1582,   468,   199,  6890,   276,   199,
            676,  2391,  2894,   199,   364,  6691,  1345,   543,   199, 14669,
            313,   423,   199, 15118,  1602,  7755,   199,  6200, 16463,  1504,
           2537,   199,  5855,  5355,  1504,   199,  3746,   542,   301,   199,
           1376, 16300,  3497,   373,  3184,   199,   608,   260,   782,  7282,
           3497,   373,  3184,   199,   364,  5514,  2087,  3497,   373,  3184,
            199,   528,  1175,  1967,  3497,   373,  3184,   199,  5392, 17145,
            199,  2050,  3477,   948,  6954,   301,   199, 20822,  6954,   301,
            199,   479,   614,  4182,  8992,  1108,   199,  1025,   449,   276,
            199,   527,  19

### DistilGPT 2 finetuning

In [38]:
import transformers
print("DistilGPT2LMHeadModel" in dir(transformers))

False


In [39]:
from transformers import AutoTokenizer, AutoModelForCausalLM


model = AutoModelForCausalLM.from_pretrained("distilgpt2")


In [40]:
model.resize_token_embeddings(len(arm_tokenizer))

Embedding(25007, 768)

In [41]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(25007, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=25007, bias=False)
)

In [42]:
device

'cuda'

In [43]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [44]:
torch.cuda.empty_cache()

In [45]:

print(torch.cuda.memory_summary(device=None, abbreviated=False))


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 251123 KiB | 251123 KiB | 251123 KiB |      0 B   |
|       from large pool | 244736 KiB | 244736 KiB | 244736 KiB |      0 B   |
|       from small pool |   6387 KiB |   6387 KiB |   6387 KiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         | 251123 KiB | 251123 KiB | 251123 KiB |      0 B   |
|       from large pool | 244736 KiB | 244736 KiB | 244736 KiB |

In [ ]:
import time

num_epochs = 1  
print_interval = 50 
batch_size = 8
max_items = 16000  # Max number of items to process
max_batches = max_items // batch_size  

model.train()
for epoch in range(num_epochs):
    start_time = time.time()
    for i, batch in enumerate(stream_dataloader(tokenized_dataset, arm_tokenizer, batch_size=batch_size)):
        if i >= max_batches:
            break  # Stop after processing max_batches

        # Move batch to the same device as the model
        inputs = {k: v.to(device) for k, v in batch.items()}

        # Forward 
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss

        # Backward 
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # print 
        if i % print_interval == 0:
            end_time = time.time()
            print(f"Epoch {epoch} | Batch {i} | Loss: {loss.item()} | Time Elapsed: {end_time - start_time} seconds")
            print("Sample Input Token IDs:", inputs["input_ids"][0][:30])  # First 30 tokens
            print("Corresponding Words:", arm_tokenizer.decode(inputs["input_ids"][0][:30]))
            start_time = time.time()  

        # stop at the end of an epoch if max_batches is reached
        if (i + 1) % max_batches == 0:
            break


Epoch 0 | Batch 0 | Loss: 9.402226448059082 | Time Elapsed: 8.042847394943237 seconds
Sample Input Token IDs: tensor([   48,    82,  6675,  2370,    69,  1716,  1389, 23122,  3879, 24877,
          587,  3200, 12813, 16731,  1081,   373,  6062,   676,   773,   587,
           13,  5526, 13239,   468, 10335,   276,   276,   330,  2498, 10577],
       device='cuda:0')
Corresponding Words: Private BankingԱրտոնությունԲիզնեսինՆերդրումային բիզնեսՎՏԲ-Հայաստան Բանկի մասինԲաժնետերեր և ներդրողն
Epoch 0 | Batch 50 | Loss: 4.5961222648620605 | Time Elapsed: 27.21561360359192 seconds
Sample Input Token IDs: tensor([10935,  6176,  8040,   306,    12,   572,   723, 13407,  1589,   279,
         6748, 13407,  1589,   279,  7017,  6522, 24368, 23354,  1568,  6208,
         1172,   874,   610,    14,   400,    13,   413, 20085,   674,  1486],
       device='cuda:0')
Corresponding Words: Բրենդան Ֆիլիպս, Նոյ խաղողի բերքահավաք խաղողի բերք մայրը որդին պոռնո Ջոնս Օճառ 3. մաս-փորված անցքեր
Epoch 0 | Batch 100

Or even push it to the [Hugging Face Hub](https://huggingface.co/models) to use that new tokenzier from anywhere. Just make sure you have your authentication token stored by executing `huggingface-cli login` in a terminal or executing the following cell:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

We are almost there, it is also necessary that you have `git lfs` installed. You can do it directly from this notebook by uncommenting the following cells:

In [ ]:
# !apt install git-lfs

In [ ]:
new_tokenizer.push_to_hub("my-new-shiny-tokenizer")

'https://huggingface.co/sgugger/my-new-shiny-tokenizer/commit/4714349893fd1aba4ed1e84910a13d631d07395a'

The tokenizer can now be reloaded on this machine with:

In [ ]:
tok = new_tokenizer.from_pretrained("my-new-tokenizer")

Or from anywhere using the repo ID, which is your namespace followed by a slash an the name you gave in the `push_to_hub` method, so for instance:

```python
tok = new_tokenizer.from_pretrained("sgugger/my-new-shiny-tokenizer")
```

Now if you want to create and a train a new tokenizer that doesn't look like anything in existence, you will need to build it from scratch using the 🤗 Tokenizers library.

## Building your tokenizer from scratch

To understand how to build your tokenizer from scratch, we have to dive a little bit more in the 🤗 Tokenizers library and the tokenization pipeline. This pipeline takes several steps:

- **Normalization**: Executes all the initial transformations over the initial input string. For example when you need to lowercase some text, maybe strip it, or even apply one of the common unicode normalization process, you will add a Normalizer.
- **Pre-tokenization**: In charge of splitting the initial input string. That's the component that decides where and how to pre-segment the origin string. The simplest example would be to simply split on spaces.
- **Model**: Handles all the sub-token discovery and generation, this is the part that is trainable and really dependent of your input data.
- **Post-Processing**: Provides advanced construction features to be compatible with some of the Transformers-based SoTA models. For instance, for BERT it would wrap the tokenized sentence around [CLS] and [SEP] tokens.

And to go in the other direction:

- **Decoding**: In charge of mapping back a tokenized input to the original string. The decoder is usually chosen according to the `PreTokenizer` we used previously.

For the training of the model, the 🤗 Tokenizers library provides a `Trainer` class that we will use.

All of these building blocks can be combined to create working tokenization pipelines. To give you some examples, we will show three full pipelines here: how to replicate GPT-2, BERT and T5 (which will give you an example of BPE, WordPiece and Unigram tokenizer).

### WordPiece model like BERT

Let's have a look at how we can create a WordPiece tokenizer like the one used for training BERT. The first step is to create a `Tokenizer` with an empty `WordPiece` model:

In [ ]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

This `tokenizer` is not ready for training yet. We have to add some preprocessing steps: the normalization (which is optional) and the pre-tokenizer, which will split inputs into the chunks we will call words. The tokens will then be part of those words (but can't be larger than that).

In the case of BERT, the normalization is lowercasing. Since BERT is such a popular model, it has its own normalizer:

In [ ]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

If you want to customize it, you can use the existing blocks and compose them in a sequence: here for instance we lower case, apply NFD normalization and strip the accents:

In [ ]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

There is also a `BertPreTokenizer` we can use directly. It pre-tokenizes using white space and punctuation:

In [ ]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

Like for the normalizer, we can combine several pre-tokenizers in a `Sequence`. If we want to have a quick look at how it preprocesses the inputs, we can call the `pre_tokenize_str` method:

In [ ]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('This', (0, 4)),
 ('is', (5, 7)),
 ('an', (8, 10)),
 ('example', (11, 18)),
 ('!', (18, 19))]

Note that the pre-tokenizer not only split the text into words but keeps the offsets, that is the beginning and start of each of those words inside the original text. This is what will allow the final tokenizer to be able to match each token to the part of the text that it comes from (a feature we use for question answering or token classification tasks).

We can now train our tokenizer (the pipeline is not entirely finished but we will need a trained tokenizer to build the post-processor), we use a `WordPieceTrainer` for that. The key thing to remember is to pass along the special tokens to the trainer, as they won't be seen in the corpus.

In [ ]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

To actually train the tokenizer, the method looks like what we used before: we can either pass some text files, or an iterator of batches of texts:

In [ ]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

Now that the tokenizer is trained, we can define the post-processor: we need to add the CLS token at the beginning and the SEP token at the end (for single sentences) or several SEP tokens (for pairs of sentences). We use a [`TemplateProcessing`](https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#tokenizers.processors.TemplateProcessing) to do this, which requires to know the IDs of the CLS and SEP token (which is why we waited for the training).

So let's first grab the ids of the two special tokens:

In [ ]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


And here is how we can build our post processor. We have to indicate in the template how to organize the special tokens with one sentence (`$A`) or two sentences (`$A` and `$B`). The `:` followed by a number indicates the token type ID to give to each part.

In [ ]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

We can check we get the expected results by encoding a pair of sentences for instance:

In [ ]:
encoding = tokenizer.encode("This is one sentence.", "With this one we have a pair.")

We can look at the tokens to check the special tokens have been inserted in the right places:

In [ ]:
encoding.tokens

['[CLS]',
 'this',
 'is',
 'one',
 'sentence',
 '.',
 '[SEP]',
 'with',
 'this',
 'one',
 'we',
 'have',
 'a',
 'pair',
 '.',
 '[SEP]']

And we can check the token type ids are correct:

In [ ]:
encoding.type_ids

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

The last piece in this tokenizer is the decoder, we use a `WordPiece` decoder and indicate the special prefix `##`:

In [ ]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

Now that our tokenizer is finished, we need to wrap it inside a Transformers object to be able to use it with the Transformers library. More specifically, we have to put it inside the class of tokenizer fast corresponding to the model we want to use, here a `BertTokenizerFast`:

In [ ]:
from transformers import BertTokenizerFast

new_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

And like before, we can use this tokenizer as a normal Transformers tokenizer, and use the `save_pretrained` or `push_to_hub` methods.

If the tokenizer you are building does not match any class in Transformers because it's really special, you can wrap it in `PreTrainedTokenizerFast`.

### BPE model like GPT-2

Let's now have a look at how we can create a BPE tokenizer like the one used for training GPT-2. The first step is to create a `Tokenizer` with an empty `BPE` model:

In [ ]:
tokenizer = Tokenizer(models.BPE())

Like before, we have to add the optional normalization (not used in the case of GPT-2) and we need to specify a pre-tokenizer before training. In the case of GPT-2, the pre-tokenizer used is a byte level pre-tokenizer:

In [ ]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

If we want to have a quick look at how it preprocesses the inputs, we can call the `pre_tokenize_str` method:

In [ ]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('This', (0, 4)),
 ('Ġis', (4, 7)),
 ('Ġan', (7, 10)),
 ('Ġexample', (10, 18)),
 ('!', (18, 19))]

We used the same default as for GPT-2 for the prefix space, so you can see that each word gets an initial `'Ġ'` added at the beginning, except the first one.

We can now train our tokenizer! This time we use a `BpeTrainer`.

In [ ]:
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

To finish the whole pipeline, we have to include the post-processor and decoder:

In [ ]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

And like before, we finish by wrapping this in a Transformers tokenizer object:

In [ ]:
from transformers import GPT2TokenizerFast

new_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

### Unigram model like Albert

Let's now have a look at how we can create a Unigram tokenizer like the one used for training T5. The first step is to create a `Tokenizer` with an empty `Unigram` model:

In [ ]:
tokenizer = Tokenizer(models.Unigram())

Like before, we have to add the optional normalization (here some replaces and lower-casing) and we need to specify a pre-tokenizer before training. The pre-tokenizer used is a `Metaspace` pre-tokenizer: it replaces all spaces by a special character (defaulting to ▁) and then splits on that character.

In [ ]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.Replace("``", '"'), normalizers.Replace("''", '"'), normalizers.Lowercase()]
)
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

If we want to have a quick look at how it preprocesses the inputs, we can call the `pre_tokenize_str` method:

In [ ]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('▁This', (0, 4)), ('▁is', (4, 7)), ('▁an', (7, 10)), ('▁example!', (10, 19))]

You can see that each word gets an initial `▁` added at the beginning, as is usually done by sentencepiece.

We can now train our tokenizer! This time we use a `UnigramTrainer`."We have to explicitely set the unknown token in this trainer otherwise it will forget it afterward.

In [ ]:
trainer = trainers.UnigramTrainer(vocab_size=25000, special_tokens=["[CLS]", "[SEP]", "<unk>", "<pad>", "[MASK]"], unk_token="<unk>")
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

To finish the whole pipeline, we have to include the post-processor and decoder. The post-processor is very similar to what we saw with BERT, the decoder is just `Metaspace`, like for the pre-tokenizer.

In [ ]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")

In [ ]:
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS]:0 $A:0 [SEP]:0",
    pair="[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)
tokenizer.decoder = decoders.Metaspace()

And like before, we finish by wrapping this in a Transformers tokenizer object:

In [ ]:
from transformers import AlbertTokenizerFast

new_tokenizer = AlbertTokenizerFast(tokenizer_object=tokenizer)

## Use your new tokenizer to train a language model!

You can either use your new tokenizer in the language modeling from scratch notebook [Link to come] or use the `--tokenizer_name` argument in the [language modeling scripts](https://github.com/huggingface/transformers/tree/master/examples/pytorch/language-modeling) to use it there to train a model from scratch.